In [23]:
import os
import pandas as pd
import numpy as np


In [24]:
def horizon_generator():
    i = 0
    while True:
        yield np.floor(5*np.sqrt(2)**i + 0.5).astype(int)
        i += 1

def algorithm_B(runtimes:np.array, solvable, s, gamma, bound=20, addition=0):
    """
        Change the algorithm of B to have a bound n like algorithm A
        and use this one to simulate algorithm A with gamma=1, and 
        that would do the trick. 

        the addition parameter is used by the madagascar algorithm to add 0.5 seconds.
    """
    print(f'runtimes : {runtimes}')
    total_runtime = 0
    if bound > len(runtimes):
        powers = np.linspace(1, len(runtimes)-1, len(runtimes)-1)
        gammas = np.power(np.array([gamma for _ in range(len(runtimes)-1)]), powers)
    else:
         powers = np.linspace(1, bound-1, bound-1)
         gammas = np.power(np.array([gamma for _ in range(bound-1)]), powers)
    
    
    gammas = np.insert(gammas, 0, 1)
    s_list = [s for _ in gammas]
    relative_runtimes = np.array([i*j for i, j in zip(s_list, gammas)]) + addition
    print(f'rel runtimes {relative_runtimes}')

    while True:
        print(f'total runtime : {total_runtime}')
        print(f'runtimes : {runtimes}')
        for idx, (rel, runtime, sol) in enumerate(zip(relative_runtimes, runtimes, solvable)):
            if rel >= runtime and sol == 1:
                total_runtime += runtime
                return total_runtime 
            elif rel >= runtime and sol == 0:
                total_runtime += runtime
                return total_runtime + algorithm_B(runtimes[idx+1:], solvable[idx+1:], s, gamma, bound)
            else:
                total_runtime += rel
                runtimes[idx] -= rel 


def algorithm_A(runtimes:list[float], solvable:list[int], bound=20, addition=0):
    """
        Implementation of algorithm A, with n parallel processes.
    """
    return algorithm_B(np.array(runtimes), solvable, 1, 1, bound, addition=addition)
    
def algorithm_C(runtimes, solvable):
    """

    """
    C_horizons = [round(5*np.sqrt(2)**i) for i in range(12)]
    C_runtimes = [runtimes[i-1] for i in C_horizons if i < len(runtimes)]
    C_horizons = [solvable[i-1] for i in C_horizons if i < len(solvable)]
    return algorithm_A(C_runtimes, C_horizons)



In [31]:
generator = horizon_generator()
sequence = np.array([0]+[next(generator) for _ in range(12)])
print(sequence)

df = pd.read_csv('./predictions/conv_grounded_results_transferLearning_4Epochs2.csv')
df.file = df.file.str.replace(pat='.pt', repl='')
df['C_horizons'] = df.prediction.apply(lambda x: list(sequence + x))
df['Impr_C_horizons'] = df.prediction.apply(lambda x: [i for i in sequence if x <= i])
print(df.head)

df.to_csv('results_testset_TransferLearning.csv', index=False)


[  0   5   7  10  14  20  28  40  57  80 113 160 226]
<bound method NDFrame.head of          domain  file  true  prediction   
0        depots  p256     8          12  \
1        depots  p255    10          11   
2        depots  p300    12          12   
3        depots  p212    10          10   
4        depots  p259     9          11   
..          ...   ...   ...         ...   
917  zenotravel  p369     6           6   
918  zenotravel  p207     9           8   
919  zenotravel  p277     6           9   
920  zenotravel  p311     9          10   
921  zenotravel  p212     9           8   

                                            C_horizons   
0    [12, 17, 19, 22, 26, 32, 40, 52, 69, 92, 125, ...  \
1    [11, 16, 18, 21, 25, 31, 39, 51, 68, 91, 124, ...   
2    [12, 17, 19, 22, 26, 32, 40, 52, 69, 92, 125, ...   
3    [10, 15, 17, 20, 24, 30, 38, 50, 67, 90, 123, ...   
4    [11, 16, 18, 21, 25, 31, 39, 51, 68, 91, 124, ...   
..                                                 